# Belief Revision Assignment

02180 Introduction to Artificial Intelligence

You are asked to implement a belief revision agent. By default, the project should run through
the following sequence of stages:
1. design and implementation of belief base;
2. design and implementation of a method for checking logical entailment (e.g., resolution-based), you should implement it yourself, without using any existing packages;
3. implementation of contraction of belief base (based on a priority order on alphas in the belief base);
4. implementation of expansion of belief base.
The output should be the resulting/new belief base.

## TODO

- [ ] Method to check logical entailment
- [ ] Contraction of belief base
- [ ] Expansion of belief base
- [ ] Use AGM postulates (lecture 11) to test your algorithm

In [348]:
from sympy import to_cnf, Equivalent
from sympy import Or, And, Not

In [349]:
def removeall(item, seq):
    return [x for x in seq if x != item]

def unique(seq):
    return list(set(seq))

def disjuncts(clause):
    return dissociate(Or, [clause])

def conjuncts(clause):
    return dissociate(And, [clause])

def associate(op, args):
    args = dissociate(op, args)
    if len(args) == 0:
        return op.identity
    elif len(args) == 1:
        return args[0]
    else:
        return op(*args)

def dissociate(op, args):
    result = []

    def collect(subargs):
        for arg in subargs:
            if isinstance(arg, op):
                collect(arg.args)
            else:
                result.append(arg)

    collect(args)
    return result

In [350]:
class BeliefBase:
    def __init__(self):
        self.beliefs = []
        self.belief_base = []

    def add_belief(self, belief) -> None:
        """Add a belief to the belief base, keeping the belief base in CNF form"""
        belief = to_cnf(belief)
        self.belief_base.append(belief)

    def pl_resolution(self, formula):
        """
        Resolution-based entailment check for base |- formula.
        """

        formula = to_cnf(formula)

        # Split base into conjuncts
        clauses = []
        for f in self.belief_base:
            clauses += conjuncts(f)
        # Add contradiction to start resolution
        clauses += conjuncts(to_cnf(~formula))

        # Special case if one clause is already False
        if False in clauses:
            return True

        result = set()
        while True:
            n = len(clauses)
            pairs = [
                (clauses[i], clauses[j])
                for i in range(n) for j in range(i + 1, n)
            ]

            for ci, cj in pairs:
                resolvents = self.pl_resolve(ci, cj)
                if False in resolvents:
                    return True
                result = result.union(set(resolvents))

            if result.issubset(set(clauses)):
                return False
            for c in result:
                if c not in clauses:
                    clauses.append(c)


    def pl_resolve(self, ci, cj):
        """
        Generate all clauses that can be obtained by applying
        the resolution rule on ci and cj.
        """

        clauses = []
        dci = disjuncts(ci)
        dcj = disjuncts(cj)

        for di in dci:
            for dj in dcj:
                # If di, dj are complementary
                if di == ~dj or ~di == dj:
                    # Create list of all disjuncts except di and dj
                    res = removeall(di, dci) + removeall(dj, dcj)
                    # Remove duplicates
                    res = unique(res)
                    # Join into new clause
                    dnew = associate(Or, res)

                    clauses.append(dnew)

        return clauses

    def __str__(self):
        return str(self.belief_base)

In [352]:
# Create an instance of BeliefBase
kb = BeliefBase()

# Add a belief to the belief base
kb.add_belief("q & p")
# belief_base.add_belief("r | s")
print(f"After adding: {kb}")

# Check if a belief is entailed by the belief base
alpha = "r"
entails = kb.pl_resolution(alpha)
print(f"entails {alpha}: {entails}")

# The problem is that we have to feed the resolution function with a discunction form and a sentence
# We also have to adapt the transformation functions to how our belief base works

After adding: [p & q]
entails r: False
